# Kamino Liquidity Analysis - Example Usage

This notebook demonstrates how to use the Kamino Liquidity Analysis tool to assess liquidation risk.

## Setup

First, import the necessary modules:

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import the analysis tool
from kamino_liquidity_analysis import (
    generate_liquidity_report,
    summarize_report,
    export_report,
)

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

# Configure plotting
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Basic Analysis - All Volatile Assets

Run a full analysis of all volatile collateral assets:

In [ ]:
# Run full analysis
df = generate_liquidity_report()

# Display basic info
print(f"Total scenarios analyzed: {len(df)}")
print(f"Successful quotes: {df['quote_success'].sum()}")
print(f"Failed quotes: {(~df['quote_success']).sum()}")
print(f"\nUnique assets: {df['asset_symbol'].nunique()}")
print(f"Assets analyzed: {', '.join(df['asset_symbol'].unique())}")

## 2. Summary Pivot Table

View price impact by asset and swap size:

In [ ]:
# Generate summary table
summary = summarize_report(df)
print("\nPrice Impact (%) by Asset and Swap Size")
print("=" * 80)
print(summary)

## 3. Detailed Results

Examine the full dataset:

In [ ]:
# Display sample rows
display_cols = [
    'asset_symbol',
    'current_price_usd',
    'current_tvl_usd',
    'swap_size_usd',
    'price_impact_pct',
    'output_usd',
    'router',
    'route_summary',
]

df[display_cols].head(20)

## 4. Filter High-Impact Scenarios

Identify scenarios with >5% price impact:

In [ ]:
# Filter to successful quotes first
successful_df = df[df['quote_success']].copy()

# Find high-impact scenarios
high_impact = successful_df[successful_df['price_impact_pct'] > 5.0]

print(f"High-impact scenarios (>5%): {len(high_impact)}")
print("\nTop 10 by price impact:")
high_impact.nlargest(10, 'price_impact_pct')[[
    'asset_symbol',
    'swap_size_usd',
    'price_impact_pct',
    'output_usd',
    'router'
]]

## 5. Risk Flags Summary

Check for automatically identified risk flags:

In [ ]:
# Count risk flags
df['has_risk_flags'] = df['risk_flags'].apply(lambda x: len(x) > 0)
risk_count = df['has_risk_flags'].sum()

print(f"Scenarios with risk flags: {risk_count}")

# Show flagged scenarios
flagged = df[df['has_risk_flags']]
if len(flagged) > 0:
    print("\nTop 10 flagged scenarios:")
    flagged[[
        'asset_symbol',
        'swap_size_usd',
        'price_impact_pct',
        'risk_flags'
    ]].head(10)

## 6. Visualization - Price Impact by Asset

Visualize price impact across different swap sizes:

In [ ]:
# Filter to successful quotes
plot_df = df[df['quote_success']].copy()

# Create swap size labels
plot_df['swap_size_label'] = plot_df['swap_size_usd'].apply(
    lambda x: f"${x/1e6:.0f}M"
)

# Plot
plt.figure(figsize=(14, 8))
sns.barplot(
    data=plot_df,
    x='swap_size_label',
    y='price_impact_pct',
    hue='asset_symbol',
)
plt.title('Price Impact by Asset and Swap Size', fontsize=16)
plt.xlabel('Swap Size', fontsize=12)
plt.ylabel('Price Impact (%)', fontsize=12)
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 7. Visualization - Liquidity Depth Curves

Plot liquidity depth for each asset:

In [ ]:
# Filter to successful quotes
plot_df = df[df['quote_success']].copy()

# Plot liquidity curves
fig, ax = plt.subplots(figsize=(14, 8))

for asset in plot_df['asset_symbol'].unique():
    asset_df = plot_df[plot_df['asset_symbol'] == asset]
    asset_df = asset_df.sort_values('swap_size_usd')
    
    ax.plot(
        asset_df['swap_size_usd'] / 1e6,  # Convert to millions
        asset_df['price_impact_pct'],
        marker='o',
        label=asset,
        linewidth=2,
    )

ax.set_xlabel('Swap Size ($M)', fontsize=12)
ax.set_ylabel('Price Impact (%)', fontsize=12)
ax.set_title('Liquidity Depth Curves by Asset', fontsize=16)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
ax.axhline(y=5, color='r', linestyle='--', alpha=0.5, label='5% Impact Threshold')
plt.tight_layout()
plt.show()

## 8. Router Analysis

See which routers are being used:

In [ ]:
# Count router usage
successful_df = df[df['quote_success']]
router_counts = successful_df['router'].value_counts()

print("Router Usage:")
print(router_counts)

# Plot
plt.figure(figsize=(10, 6))
router_counts.plot(kind='bar')
plt.title('Jupiter Router Usage', fontsize=16)
plt.xlabel('Router', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. TVL vs Swap Size Analysis

Compare TVL to largest tested swap size:

In [ ]:
# Get max swap size per asset
max_swap_df = df.groupby('asset_symbol').agg({
    'current_tvl_usd': 'first',
    'swap_size_usd': 'max',
}).reset_index()

max_swap_df['tvl_to_swap_ratio'] = (
    max_swap_df['current_tvl_usd'] / max_swap_df['swap_size_usd']
)

print("TVL vs Max Tested Swap Size:")
print(max_swap_df.sort_values('tvl_to_swap_ratio'))

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))
x = range(len(max_swap_df))
width = 0.35

ax.bar(
    [i - width/2 for i in x],
    max_swap_df['current_tvl_usd'] / 1e6,
    width,
    label='TVL'
)
ax.bar(
    [i + width/2 for i in x],
    max_swap_df['swap_size_usd'] / 1e6,
    width,
    label='Max Test Size'
)

ax.set_xlabel('Asset', fontsize=12)
ax.set_ylabel('Amount ($M)', fontsize=12)
ax.set_title('TVL vs Max Tested Swap Size', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels(max_swap_df['asset_symbol'], rotation=45)
ax.legend()
plt.tight_layout()
plt.show()

## 10. Export Results

Save the analysis to CSV:

In [ ]:
# Export to CSV
output_file = 'kamino_liquidity_analysis.csv'
export_report(df, output_file, include_failed=True)
print(f"Results exported to: {output_file}")

## 11. Custom Analysis - Specific Assets

Run analysis on specific assets only:

In [ ]:
# Analyze only SOL-based assets
sol_assets = ['SOL', 'MSOL', 'JITOSOL']

sol_df = generate_liquidity_report(asset_filter=sol_assets)

print(f"Analyzed {len(sol_df)} scenarios for SOL-based assets")
summarize_report(sol_df)

## 12. Custom Swap Sizes

Test with custom swap sizes:

In [ ]:
# Test smaller, more granular swap sizes
custom_sizes = [
    500_000,      # $500K
    1_000_000,    # $1M
    2_500_000,    # $2.5M
    5_000_000,    # $5M
    7_500_000,    # $7.5M
    10_000_000,   # $10M
]

custom_df = generate_liquidity_report(
    swap_sizes_usd=custom_sizes,
    asset_filter=['SOL']  # Just SOL for speed
)

# Plot the more granular curve
plot_df = custom_df[custom_df['quote_success']].copy()
plt.figure(figsize=(10, 6))
plt.plot(
    plot_df['swap_size_usd'] / 1e6,
    plot_df['price_impact_pct'],
    marker='o',
    linewidth=2,
)
plt.xlabel('Swap Size ($M)', fontsize=12)
plt.ylabel('Price Impact (%)', fontsize=12)
plt.title('SOL Liquidity Depth (Granular)', fontsize=16)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. Running full liquidity analysis
2. Generating summary tables
3. Filtering and analyzing results
4. Identifying risk scenarios
5. Visualizing liquidity depth
6. Analyzing routers and routes
7. Comparing TVL to swap sizes
8. Custom asset and swap size analysis

Use these techniques to assess liquidation risk and make informed decisions about position sizing and risk management.